# 03-getting-started-langfuse

Langfuse の基本事項を試すためのノートブックです。

In [ ]:
import os
from dotenv import load_dotenv
from operator import itemgetter

from langchain_cohere import ChatCohere
from langchain_community.llms.oci_generative_ai import OCIGenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.messages import HumanMessage, ChatMessage

from langfuse import Langfuse
from langfuse.callback import CallbackHandler

In [ ]:
load_dotenv()

# Cohere
api_key = os.getenv("COHERE_API_KEY")

# OCI
compartment_id = os.getenv("COMPARTMENT_ID")
endpoint = os.getenv("GENAI_ENDPOINT")
# Langfuse
secret_key = os.getenv("LANGFUSE_SECRET_KEY")
public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_host = os.getenv("LANGFUSE_HOST")

langfuse_handler = CallbackHandler(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

## Tracing

### OCI Generative AI Service

OCI Generative AI Service を LangChain から操作する場合は、Langfuse の CallbackHandler に対応していないので、モンキーパッチを適用する

In [ ]:
def get_model_name(self):
    return self.model_id

In [ ]:
OCIGenAI.model_name = property(get_model_name)

In [ ]:
oci_genai_client = OCIGenAI(
    auth_type="API_KEY",
    auth_profile="CHICAGO",
    model_id="cohere.command",
    compartment_id=compartment_id,
    service_endpoint=endpoint,
    model_kwargs={
        "max_tokens": 512
    }
)

In [ ]:
messages = [HumanMessage(content="OCHaCafeとはなんですか？")]
res = oci_genai_client.invoke(messages, config={"callbacks": [langfuse_handler]})
print(res)

In [ ]:
prompt1 = ChatPromptTemplate.from_template("What is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("What country is the city {city} in? respond in {language}")

chain1 = prompt1 | oci_genai_client | StrOutputParser()
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | oci_genai_client
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "Spanish"}, config={"callbacks": [langfuse_handler]})

## Cohere

In [ ]:
chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=api_key
)

In [ ]:
messages = [HumanMessage(content="OCHaCafeとはなんですか？")]
res = chat.invoke(messages, config={"callbacks": [langfuse_handler]})
print(res)

In [ ]:
prompt1 = ChatPromptTemplate.from_template("What is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("What country is the city {city} in? respond in {language}")

chain1 = prompt1 | chat | StrOutputParser()
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | chat
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "Japanese"}, config={"callbacks": [langfuse_handler]})

## Score

### OCI Generative AI

In [ ]:
messages = [HumanMessage(content="OCHaCafeとはなんですか？")]
res = oci_genai_client.invoke(messages, config={"callbacks": [langfuse_handler]})
print(res)

In [ ]:
trace_id = langfuse_handler.get_trace_id()
print("trace_id: ", trace_id)

In [ ]:
# Add score
langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

trace = langfuse.score(
    trace_id=trace_id,
    value=0.1,
    name="user-feedback",
    comment="be contrary to fact"
)